# **Métodos Formais em Engenharia de Software - SMT solving**

**Nome:** Paulo Silva Sousa

**Número:** PG47556

**Curso:** Mestrado em Engenharia Informática

<br>

# **Exercício 1 - Futoshiki Puzzle**

Futoshiki é um puzzle lógico japonês jogado num tabuleiro N×N , onde são assinaladas restrições de desigualdade entre entre algumas posições contiguas do tabuleiro.

O objetivo é colocar os números 1..N de forma a que cada número não apareça repetido em cada linha nem em cada coluna do tabuleiro, e que as relações de desigualdade assinaladas sejam respeitadas. 

Alguns números podem estar fixos no tabuleiro inicial. Pode ver mais informações
sobre o puzzle em http://en.wikipedia.org/wiki/Futoshiki e http://www.brainbashers.com/futoshiki.asp

Desenvolva um programa em Phyton para resolver este jogo como auxílio de um SMT solver.

• Input: a configuração do tabuleiro inicial deverá ser fornecida num ficheiro de texto, em formato que entendam adquado para o descrever. Como alternativa pode receber o texto do tabuleiro diretamente numa string.

• Output: a solução do puzzle deverá ser impressa no ecrã.

Como input do nosso programa utilizamos um ficheiro de texto divido em 4 partes.

Na primeira linha é apresentado o tamanho N do tabuleiro.

Nas próximas N linhas são apresentados os valores das casas do tabuleiro, em que 0 é apresentado se esta estiver vazia.

Nas seguintes N linhas são apresentadas as restrições horizontais, sendo apresentado o valor 0 se não houver restrições, 1 se for maior e -1 se for menor.

Por último, nas restantes N-1 linhas são apresentadas as restrições verticais com os mesmos valores que as horizontais.

De seguida temos um exemplo de um tabuleiro de tamanho 4.

In [ ]:
4
0 0 0 3
2 0 0 0
0 0 0 0
0 0 0 0
0 0 0
0 1 0
0 0 0
0 -1 0
0 0 0 0
1 0 0 0
0 0 0 -1

Para instalar o solver, utilizamos o pip.

In [ ]:
!pip install z3-solver

     |████████████████████████████████| 33.0 MB 18 kB/s 


De seguida temos o programa em Python para resolver o Futoshiki Puzzle com o auxílio de um SMT solver.

In [ ]:
from z3 import *

# Parser

f = open("table.txt")

n = int(f.readline())

instance = []
restrictions_hor = []
restrictions_ver = []

for i in range(n):
  line = f.readline()
  aux = line.strip().split(" ")
  aux2 = [int(x) for x in aux]
  instance.append(aux2)

for i in range(n):
  line = f.readline()
  aux = line.strip().split(" ")
  aux2 = [int(x) for x in aux]
  restrictions_hor.append(aux2)

for i in range(n-1):
  line = f.readline()
  aux = line.strip().split(" ")
  aux2 = [int(x) for x in aux]
  restrictions_ver.append(aux2)

# Solver

s = Solver()
X = []

# Celulas são inteiros
for i in range(n):
  line = []
  for j in range(n):
    line.append(Int("x_%s_%s" % (i+1, j+1)))
  X.append(line)

# Celulas entre 1 e n
for i in range(n):
  for j in range(n):
    s.add(And(X[i][j] <= n),X[i][j] >= 1)

# Linhas sem celulas iguais
for i in range(n):
    s.add(Distinct(X[i]))

# Colunas sem celulas iguais
for j in range(n):
  s.add(Distinct( [ X[i][j] for i in range(n)] ))

# Restrições horizontais
for i in range(n):
  for j in range(n-1):
    if restrictions_hor[i][j] == 1:
      s.add(X[i][j] > X[i][j+1])
    elif restrictions_hor[i][j] == -1:
      s.add(X[i][j] < X[i][j+1])

# Restrições verticais
for i in range(n-1):
  for j in range(n):
    if restrictions_ver[i][j] == 1:
      s.add(X[i][j] > X[i+1][j])
    elif restrictions_ver[i][j] == -1:
      s.add(X[i][j] < X[i+1][j])

for i in range(n):
  for j in range(n):
    s.add(If(instance[i][j] == 0,True,X[i][j] == instance[i][j]))

if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(n) ]
          for i in range(n) ]
    print_matrix(r)
else:
    print("failed to solve")

[[4, 2, 1, 3], [2, 4, 3, 1], [1, 3, 4, 2], [3, 1, 2, 4]]
